In [1]:
import gzip
import csv
import random
import math

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

from collections import defaultdict

#Collect Data from file
userRecipeDataFULL = []
didCookFULL = []

usersPerRecipeTotal = defaultdict(set) 
recipesPerUserTotal = defaultdict(set)

count = 0

# - read from trainInteractions.csv.gz
for u,r,_ in readCSV("trainInteractions.csv.gz"):
    if count >100000:
        break
    # Create Training set (small to start)
    userRecipeDataFULL.append([u,r])
    usersPerRecipeTotal[r].add(u)
    recipesPerUserTotal[u].add(r)
    didCookFULL.append(1)
    count +=1 
    

In [2]:
recipes = list(usersPerRecipeTotal)

newUserRecipeData = []
for u,r in userRecipeDataFULL:
    newRecipe = None
    while newRecipe == None or newRecipe in recipesPerUserTotal[u]:
        newRecipe = random.choice(recipes)
    newUserRecipeData.append([u,newRecipe])
    didCookFULL.append(0)
userRecipeDataFULL.extend(newUserRecipeData)


randomIndices = list(range(len(userRecipeDataFULL)))
random.shuffle(randomIndices)


userRecipeData_Suffled = []
didCook_Suffled = []

for ri in randomIndices:
    userRecipeData_Suffled.append(userRecipeDataFULL[ri])
    didCook_Suffled.append(didCookFULL[ri])
    
userRecipeDataFULL=userRecipeData_Suffled
didCookFULL=didCook_Suffled

CookPerUserRecipeFULL = defaultdict(int)

for i,(u,r) in enumerate(userRecipeDataFULL):
    CookPerUserRecipeFULL[(u,r)] = didCookFULL[i]


In [3]:
import math
def Cosine(i1, i2):
    # Between two items
    inter = usersPerRecipeTotal[i1].intersection(usersPerRecipeTotal[i2]) 
    numer = sum([CookPerUserRecipeFULL[(u,i1)]*CookPerUserRecipeFULL[(u,i2)] for u in inter])
    norm1 = sum([CookPerUserRecipeFULL[(u,i1)]**2 for u in usersPerRecipeTotal[i1]])
    norm2 = sum([CookPerUserRecipeFULL[(u,i2)]**2 for u in usersPerRecipeTotal[i2]])
    denom = math.sqrt(norm1) * math.sqrt(norm2)
    if denom == 0: return 0 # If one of the two items has noratings
    return numer / denom

In [4]:
def cosineFeature(u,i):
    sims = []
    for i2 in recipesPerUserTotal[u]:
        if i2 != i:
            sims.append(Cosine(i, i2))
    return max(sims) if len(sims)>0 else 0

In [5]:
def cosineFeatures(uiPairs):
    features = []
    for u,i in uiPairs:
        features.append([1, cosineFeature(u,i)])
    return features

In [6]:
cosineFeats = cosineFeatures(userRecipeDataFULL)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
modelC = LogisticRegression().fit(cosineFeats, didCookFULL)
predC = modelC.predict(cosineFeats)
accuracy_score(predC,didCookFULL)

0.895261047389526

In [8]:
stubUserRecipe = []

for l in open("stub_Made.txt"):
    if l.startswith("user_id"):
        continue
    u, r = l.strip().split('-')
    stubUserRecipe.append([u, r])

In [9]:
# Predict Stub
featureVectorsS = cosineFeatures(stubUserRecipe)
predictions = modelC.predict(featureVectorsS)

In [10]:
# Save Stub Predictions to File
predictionsFile = open("predictions_Made.txt", 'w')
predictionsFile.write("user_id-recipe_id,prediction\n")

for i,(u,r) in enumerate(stubUserRecipe):
    predictionsFile.write(u + '-' + r + ","+str(predictions[i])+"\n")

predictionsFile.close()